## Real online dating profile download script

The below script has been sourced, with amendments, from the following repository: https://github.com/gsuareztangil/automatic-romancescam-digger. 

In [2]:
import os
import re
import json
import time
import hashlib
import random
from bs4 import BeautifulSoup
from urllib.request import urlopen
from tqdm import tqdm 
import csv
import pandas as pd

IMAGEDIR='images_real'
PROFILES='dating_real'

In [3]:
iurlrx = re.compile('.* background-image: url\(([^\)]+)\)')

remap = {'I am' : 'gender',
         'Age' : 'age',
         'City' : 'location',
         'Marital status' : 'status',
         'Username' : 'username',
         'Ethnicity' : 'ethnicity',
         'Occupation' : 'occupation',
         'About me' : 'description',
         'My match\'s age' : 'match_age',
         'Children' : 'children',
         'Sexual Orientation' : 'orientation',
         'Religion' : 'religion',
         'Do you smoke' : 'smoking',
         'Do you drink' : 'drinking',
         'Here for' : 'intent'}

In [4]:
def save_image(url):
    """ Take a URL, generate a unique filename, save 
        the image to said file and return the filename."""
    
    ext = url.split('.')[-1]
    filename = os.getcwd()+os.sep+IMAGEDIR+os.sep+hashlib.md5(url.encode('utf-8')).hexdigest()+'.'+ext
    
    if os.path.exists(filename):
        return filename
    try:
        content = urlopen(url).read()
        f = open(filename,'wb') 
        f.write(content)
        f.close()
    except e:
        print(e)
        return None
    return filename 

In [5]:
def scrape_profile(inhandle, outfile):
    """Scrape an input scamdiggers page for the profile content
    of the scammer. """
    
    # Read file.
    soup = BeautifulSoup(inhandle.read(), 'html.parser')

    pfnode = soup.find('div', {'class':'profile-BASE_CMP_UserViewWidget'})
    avnode = soup.find(id='avatar_console_image')

    # Pull the provided profile data out.
    rows = pfnode.findAll('tr')
    labels = {}
    for row in rows:
        lab = row.find('td',{'class':'ow_label'})
        val = row.find('td',{'class':'ow_value'})
        if lab:
            labels[lab.get_text()] = val.get_text().strip()

    profile = {}

    # Populate our own profile structure.
    for lab in remap:
        if lab in labels:
            profile[remap[lab]] = labels[lab]
        else:
            profile[remap[lab]] = "-"
  
    # Tweak for consistency.
    profile['gender'] = profile['gender'].lower()

    # Extract avatar image.
    img = iurlrx.match(avnode.attrs['style']).group(1)
    profile['images'] = [save_image(img)]

    # Save output.
    json.dump(profile, open(outfile,'w'))

In [6]:
def enumerate_profiles(inhandle):
    """ Extract all the profile page links from
    this index page. """
    
    soup = BeautifulSoup(inhandle.read(), 'html.parser')

    urls = []
    for node in soup.findAll('div',  {'class':'ow_user_list_data'}):
        try:
            urls.append(node.find('a')['href'])
        except:
            pass
    return urls

In [8]:
def activity_profiles(inhandle):
    """ Extract all the latest activity from
    this index page. """
    
    soup = BeautifulSoup(inhandle.read(), 'html.parser')
    activity = []
    
    for node in soup.findAll('div',  {'class':'ow_user_list_data'}):
        try:
            name = node.find('a')['href']
            last_active = node.find('span', {"class":"ow_remark"}).get_text()
            activity.append([name, last_active])
        except:
            pass
    return activity

In [9]:
def scrape(urls):
    
    for url in tqdm(urls):
        uid = url[0][34:]
        outfile=os.getcwd()+os.sep+PROFILES+os.sep+uid+'.json'
        
        jitter = random.choice([0,1])
        try:
            urlhandle = urlopen(url[0])
            scrape_profile(urlhandle, outfile)
            time.sleep(1+jitter)
        except Exception as e:
            print("Exception when handling {}".format(url))
            print(e)

    print("Scraping complete.")

In [10]:
# Store the latest activity timestamps in a csv format. 

urls = []
activity_log = []
urlstr="http://datingnmore.com/site/users/latest?page={}" 

start_time = time.perf_counter()
print("Begin URL harvesting.")

for i in range(1,3245): 
    url = urlstr.format(i)
    jitter = random.choice([0,1])
    try:
        urlhandle = urlopen(url)
        latest_activity = activity_profiles(urlhandle)
        activity_log += latest_activity
        time.sleep(1+jitter)
    except Exception as e:
        print("Exception when handling {}".format(url))
        print(e)
        break

print("Harvesting complete. {} URLs to scrape.".format(len(urls)))
print(time.perf_counter()-start_time)

pd.DataFrame(activity_log, columns = ["URL", "Last_activity"]).T.to_csv("activity_list.csv")

Begin URL harvesting.
Harvesting complete. 0 URLs to scrape.
11441.6574


In [10]:
# Load in the urls.
# This allows the scraping to be done in batches.

urls = pd.read_csv("URL_list.csv")
urls = urls.values.tolist()

In [11]:
# Run the real profile download. Use the urls list indices to specify the profiles to download. 

start_time = time.perf_counter()
print(start_time)
scrape(urls[33750:40000])

print(time.perf_counter()-start_time)

15.8248448


 15%|███████████▌                                                                 | 936/6250 [41:53<3:08:42,  2.13s/it]

Exception when handling ['https://datingnmore.com/site/user/issis']
'NoneType' object has no attribute 'findAll'


 23%|█████████████████▎                                                        | 1460/6250 [1:05:29<3:14:27,  2.44s/it]

Exception when handling ['https://datingnmore.com/site/user/Angelmar']
'NoneType' object has no attribute 'findAll'


...

Exception when handling ['https://datingnmore.com/site/user/Poly97']
'NoneType' object has no attribute 'findAll'


 92%|█████████████████████████████████████████████████████████████████████▊      | 5742/6250 [4:19:22<18:22,  2.17s/it]

Exception when handling ['https://datingnmore.com/site/user/Sladjaphx']
'NoneType' object has no attribute 'findAll'


100%|████████████████████████████████████████████████████████████████████████████| 6250/6250 [4:42:46<00:00,  2.71s/it]

Scraping complete.
16966.2554971
